In [1]:
import pandas as pd
import numpy as np

In [51]:
class MyLineReg():
    def __init__ (self, learning_rate=0.000421, n_iter=800, weights=None, metric=None, reg=None, l1_coef=0, l2_coef=0):
        self.learn_rate = learning_rate
        self.n_iter = n_iter
        self.weights = weights
        self.metric = metric
        self.final_metric = None
        self.reg = reg
        self.l1_coef = l1_coef
        self.l2_coef = l2_coef

    def calculate_metric(self, y, y_hat):
        """Calculates the specified metric."""
        if self.metric == 'mae':
            return np.mean(np.abs(y_hat - y))
        elif self.metric == 'mse':
            return np.mean(np.power(y_hat - y, 2))
        elif self.metric == 'rmse':
            return np.sqrt(np.mean(np.power(y_hat - y, 2)))
        elif self.metric == 'mape':
            return np.mean(np.abs((y - y_hat) / y)) * 100
        elif self.metric == 'r2':
            ss_res = np.sum(np.power(y - y_hat, 2))
            ss_tot = np.sum(np.power(y - np.mean(y), 2))
            return 1 - (ss_res / ss_tot)
        else:
            return None

    def calculate_reg_grad(self, weights):
        """Calculates regularization"""
        if self.reg == "l1":
            return np.vectorize(lambda x: 1 if x > 0 else (-1 if x < 0 else 0) )(weights) * self.l1_coef
        elif self.reg == "l2":
            return 2 * self.l2_coef * weights
        elif self.reg == "elasticnet":
            return np.vectorize(lambda x: 1 if x > 0 else (-1 if x < 0 else 0) )(weights) * self.l1_coef + 2 * self.l2_coef * weights
        else:
            return 0
            
    def calculate_reg_mse(self, weights):
        if self.reg == "l1":
            return np.sum(np.abs(weights))*self.l1_coef
        elif self.reg == "l2":
            return np.sum(np.power(weights, 2))*self.l2_coef
        elif self.reg == "elasticnet":
            return np.sum(np.abs(weights))*self.l1_coef + np.sum(np.power(weights, 2))*self.l2_coef
        else:
            return 0
        
    def fit(self, X, y, verbose=False):
        n_obs, n_feat = X.shape
        X_ = pd.concat([pd.Series(np.ones(n_obs)), X], axis=1)
        self.weights = np.ones(n_feat+1)
        
        if verbose:
            y_hat = X_.dot(self.weights)
            mse = np.mean(np.power(y_hat - y,2)) + self.calculate_reg_mse(self.weights)
            start_metric = self.calculate_metric(y, y_hat)
            print(f"start | loss: {mse:.2f} | {self.metric}: {start_metric:.2f}")

        for i in range(self.n_iter):
            y_hat = X_.dot(self.weights)
            mse = np.mean(np.power(y_hat - y,2)) + self.calculate_reg_mse(self.weights)
            grad = (2 * np.dot( (y_hat - y), X_))/n_obs + self.calculate_reg_grad(self.weights)
            metric_value = self.calculate_metric(y, y_hat)
            self.weights -= grad*self.learn_rate

            if verbose and (i+1) % verbose == 0:
                print(f"{i+1} | {mse:.2f} | {self.metric}: {metric_value:.2f}")
        y_hat = X_.dot(self.weights)
        self.final_metric = self.calculate_metric(y, y_hat)
    
    def predict(self, X_test):
        n_obs_test = X_test.shape[0]
        X_test_ = pd.concat([pd.Series(np.ones(n_obs_test)), X_test], axis=1)
        Y_predicted = X_test_.dot(self.weights)
        return Y_predicted.sum()

    
    def get_coef(self):
        return self.weights[1:]
            

    def get_best_score(self):
        return self.final_metric
        
    def __str__ (self):
        return f'MyLineReg class: n_iter={self.n_iter}, learning_rate={self.learn_rate}'

In [35]:
X = pd.DataFrame({"feat_1": [1,2,4,6],
                 "feat_2": [45, 2,34,78]})
y = pd.Series([23, 28, 56, 73])

In [67]:
cl = MyLineReg(metric="mape", reg="elasticnet", l1_coef=0.5)

In [69]:
cl.fit(X=X,y=y)

In [71]:
cl.predict(pd.DataFrame({"feat_1": [4,6,8,9],
                 "feat_2": [45, 2,34,78]}))

304.54681384433854

In [59]:
cl.get_best_score()

9.820119132376405